In [1]:
from torch_geometric.datasets import MoleculeNet

data = MoleculeNet(root=".", name="ESOL")
print(data)

print("Dataset type: ", type(data))
print("Dataset features:", data.num_features)
print("Dataset target:", data.num_classes)
print("Dataset sample:", data[0])
print("Dataset Size:", len(data))

Processing...


ESOL(1128)
Dataset type:  torch_geometric.datasets.molecule_net.MoleculeNet
Dataset features: 9
Dataset target: 734
Dataset sample: Data(x=[32, 9], edge_index=[2, 68], edge_attr=[68, 3], y=[1, 1], smiles='OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O ')
Dataset Size: 1128


Done!


In [2]:
# 基于SMILES字符串，生成分子式，写入mol.png，即图1
from rdkit import Chem
from rdkit.Chem import Draw

molecule = Chem.MolFromSmiles(data[0]["smiles"])
Draw.MolToFile(molecule, "mol.png")
print(f"Type of molecule: {type(molecule)}")

Type of molecule: <class 'rdkit.Chem.rdchem.Mol'>


In [6]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gmp, global_max_pool as gap
import torch.nn as nn
embedding_size = 64

class GCN(nn.Module):
    def __init__(self):
        # Initialization
        super().__init__()
        torch.manual_seed(42)

        # GCN Layers，第1层网络将特征转化为embedding
        self.initial_conv = GCNConv(data.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # output layer，输出是一个线性层，将128维向量转成预测值，该值是个标量，维度和y相同，X2是因为下面要cat两个pooling
        self.out = nn.Linear(embedding_size*2, 1)

    def forward(self, x , edge_index, batch_index):
        # 1st Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = torch.tanh(hidden)

        # Other layers
        hidden = self.conv1(hidden, edge_index)
        hidden = torch.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = torch.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = torch.tanh(hidden)
        print('hidden:',hidden.shape)
        # Global pooling
        hidden = torch.cat((gmp(hidden, batch_index), gap(hidden, batch_index)), dim=1)##############64+64
        
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)

GCN(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)


In [7]:
from torch_geometric.loader import DataLoader

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

data_size = len(data)
batch_size = 64
train_loader = DataLoader(data[:int(data_size*0.8)], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data[int(data_size*0.8):], batch_size=batch_size)

In [8]:
def train(data):
    for batch in train_loader:
        optimizer.zero_grad()

        pred, embeding = model(batch.x.float(), batch.edge_index, batch.batch)
        loss = torch.sqrt(loss_fn(pred, batch.y))

        loss.backward()
        optimizer.step()

    return loss, embeding

print("Start training ...")
losses = []
for epoch in range(1001):
    loss, h = train(data)
    losses.append(loss)
    if epoch%100 == 0:
        print(f"Epoch {epoch} | Training loss {loss}")

Start training ...
hidden: torch.Size([913, 64])
hidden: torch.Size([829, 64])
hidden: torch.Size([873, 64])
hidden: torch.Size([790, 64])
hidden: torch.Size([859, 64])
hidden: torch.Size([971, 64])
hidden: torch.Size([834, 64])
hidden: torch.Size([805, 64])
hidden: torch.Size([841, 64])
hidden: torch.Size([802, 64])
hidden: torch.Size([846, 64])
hidden: torch.Size([867, 64])
hidden: torch.Size([826, 64])
hidden: torch.Size([855, 64])
hidden: torch.Size([86, 64])
Epoch 0 | Training loss 4.9329962730407715
hidden: torch.Size([796, 64])
hidden: torch.Size([849, 64])
hidden: torch.Size([939, 64])
hidden: torch.Size([893, 64])
hidden: torch.Size([873, 64])
hidden: torch.Size([817, 64])
hidden: torch.Size([870, 64])
hidden: torch.Size([817, 64])
hidden: torch.Size([944, 64])
hidden: torch.Size([819, 64])
hidden: torch.Size([834, 64])
hidden: torch.Size([774, 64])
hidden: torch.Size([870, 64])
hidden: torch.Size([824, 64])
hidden: torch.Size([78, 64])
hidden: torch.Size([827, 64])
hidden: to

hidden: torch.Size([749, 64])
hidden: torch.Size([826, 64])
hidden: torch.Size([867, 64])
hidden: torch.Size([840, 64])
hidden: torch.Size([859, 64])
hidden: torch.Size([934, 64])
hidden: torch.Size([79, 64])
hidden: torch.Size([899, 64])
hidden: torch.Size([978, 64])
hidden: torch.Size([820, 64])
hidden: torch.Size([792, 64])
hidden: torch.Size([839, 64])
hidden: torch.Size([811, 64])
hidden: torch.Size([859, 64])
hidden: torch.Size([823, 64])
hidden: torch.Size([813, 64])
hidden: torch.Size([851, 64])
hidden: torch.Size([877, 64])
hidden: torch.Size([802, 64])
hidden: torch.Size([993, 64])
hidden: torch.Size([747, 64])
hidden: torch.Size([93, 64])
hidden: torch.Size([864, 64])
hidden: torch.Size([760, 64])
hidden: torch.Size([794, 64])
hidden: torch.Size([856, 64])
hidden: torch.Size([792, 64])
hidden: torch.Size([883, 64])
hidden: torch.Size([887, 64])
hidden: torch.Size([860, 64])
hidden: torch.Size([805, 64])
hidden: torch.Size([918, 64])
hidden: torch.Size([852, 64])
hidden: torc

hidden: torch.Size([885, 64])
hidden: torch.Size([864, 64])
hidden: torch.Size([783, 64])
hidden: torch.Size([832, 64])
hidden: torch.Size([865, 64])
hidden: torch.Size([870, 64])
hidden: torch.Size([853, 64])
hidden: torch.Size([849, 64])
hidden: torch.Size([879, 64])
hidden: torch.Size([777, 64])
hidden: torch.Size([846, 64])
hidden: torch.Size([888, 64])
hidden: torch.Size([918, 64])
hidden: torch.Size([776, 64])
hidden: torch.Size([112, 64])
hidden: torch.Size([903, 64])
hidden: torch.Size([958, 64])
hidden: torch.Size([768, 64])
hidden: torch.Size([877, 64])
hidden: torch.Size([768, 64])
hidden: torch.Size([854, 64])
hidden: torch.Size([919, 64])
hidden: torch.Size([833, 64])
hidden: torch.Size([908, 64])
hidden: torch.Size([842, 64])
hidden: torch.Size([831, 64])
hidden: torch.Size([878, 64])
hidden: torch.Size([824, 64])
hidden: torch.Size([771, 64])
hidden: torch.Size([63, 64])
hidden: torch.Size([915, 64])
hidden: torch.Size([763, 64])
hidden: torch.Size([764, 64])
hidden: tor

hidden: torch.Size([829, 64])
hidden: torch.Size([804, 64])
hidden: torch.Size([777, 64])
hidden: torch.Size([75, 64])
hidden: torch.Size([905, 64])
hidden: torch.Size([945, 64])
hidden: torch.Size([803, 64])
hidden: torch.Size([757, 64])
hidden: torch.Size([894, 64])
hidden: torch.Size([916, 64])
hidden: torch.Size([759, 64])
hidden: torch.Size([781, 64])
hidden: torch.Size([841, 64])
hidden: torch.Size([856, 64])
hidden: torch.Size([871, 64])
hidden: torch.Size([844, 64])
hidden: torch.Size([905, 64])
hidden: torch.Size([862, 64])
hidden: torch.Size([58, 64])
hidden: torch.Size([861, 64])
hidden: torch.Size([752, 64])
hidden: torch.Size([820, 64])
hidden: torch.Size([813, 64])
hidden: torch.Size([807, 64])
hidden: torch.Size([921, 64])
hidden: torch.Size([913, 64])
hidden: torch.Size([860, 64])
hidden: torch.Size([866, 64])
hidden: torch.Size([707, 64])
hidden: torch.Size([850, 64])
hidden: torch.Size([870, 64])
hidden: torch.Size([1000, 64])
hidden: torch.Size([904, 64])
hidden: tor

hidden: torch.Size([777, 64])
hidden: torch.Size([861, 64])
hidden: torch.Size([784, 64])
hidden: torch.Size([871, 64])
hidden: torch.Size([887, 64])
hidden: torch.Size([807, 64])
hidden: torch.Size([868, 64])
hidden: torch.Size([928, 64])
hidden: torch.Size([885, 64])
hidden: torch.Size([1017, 64])
hidden: torch.Size([675, 64])
hidden: torch.Size([779, 64])
hidden: torch.Size([90, 64])
hidden: torch.Size([800, 64])
hidden: torch.Size([1005, 64])
hidden: torch.Size([810, 64])
hidden: torch.Size([808, 64])
hidden: torch.Size([869, 64])
hidden: torch.Size([915, 64])
hidden: torch.Size([785, 64])
hidden: torch.Size([917, 64])
hidden: torch.Size([846, 64])
hidden: torch.Size([866, 64])
hidden: torch.Size([785, 64])
hidden: torch.Size([891, 64])
hidden: torch.Size([829, 64])
hidden: torch.Size([793, 64])
hidden: torch.Size([78, 64])
hidden: torch.Size([927, 64])
hidden: torch.Size([820, 64])
hidden: torch.Size([897, 64])
hidden: torch.Size([827, 64])
hidden: torch.Size([788, 64])
hidden: to

...

hidden: torch.Size([867, 64])
hidden: torch.Size([851, 64])
hidden: torch.Size([845, 64])
hidden: torch.Size([901, 64])
hidden: torch.Size([86, 64])
hidden: torch.Size([999, 64])
hidden: torch.Size([825, 64])
hidden: torch.Size([879, 64])
hidden: torch.Size([868, 64])
hidden: torch.Size([854, 64])
hidden: torch.Size([827, 64])
hidden: torch.Size([814, 64])
hidden: torch.Size([786, 64])
hidden: torch.Size([863, 64])
hidden: torch.Size([851, 64])
hidden: torch.Size([815, 64])
hidden: torch.Size([858, 64])
hidden: torch.Size([801, 64])
hidden: torch.Size([876, 64])
hidden: torch.Size([81, 64])
hidden: torch.Size([791, 64])
hidden: torch.Size([901, 64])
hidden: torch.Size([876, 64])
hidden: torch.Size([878, 64])
hidden: torch.Size([919, 64])
hidden: torch.Size([793, 64])
hidden: torch.Size([915, 64])
hidden: torch.Size([718, 64])
hidden: torch.Size([856, 64])
hidden: torch.Size([882, 64])
hidden: torch.Size([862, 64])
hidden: torch.Size([911, 64])
hidden: torch.Size([812, 64])
hidden: torc

hidden: torch.Size([866, 64])
hidden: torch.Size([889, 64])
hidden: torch.Size([841, 64])
hidden: torch.Size([978, 64])
hidden: torch.Size([885, 64])
hidden: torch.Size([818, 64])
hidden: torch.Size([776, 64])
hidden: torch.Size([807, 64])
hidden: torch.Size([791, 64])
hidden: torch.Size([911, 64])
hidden: torch.Size([82, 64])
hidden: torch.Size([869, 64])
hidden: torch.Size([808, 64])
hidden: torch.Size([884, 64])
hidden: torch.Size([935, 64])
hidden: torch.Size([836, 64])
hidden: torch.Size([767, 64])
hidden: torch.Size([840, 64])
hidden: torch.Size([923, 64])
hidden: torch.Size([768, 64])
hidden: torch.Size([775, 64])
hidden: torch.Size([878, 64])
hidden: torch.Size([879, 64])
hidden: torch.Size([911, 64])
hidden: torch.Size([855, 64])
hidden: torch.Size([69, 64])
hidden: torch.Size([779, 64])
hidden: torch.Size([831, 64])
hidden: torch.Size([840, 64])
hidden: torch.Size([850, 64])
hidden: torch.Size([905, 64])
hidden: torch.Size([915, 64])
hidden: torch.Size([946, 64])
hidden: torc

hidden: torch.Size([884, 64])
hidden: torch.Size([755, 64])
hidden: torch.Size([835, 64])
hidden: torch.Size([94, 64])
hidden: torch.Size([854, 64])
hidden: torch.Size([786, 64])
hidden: torch.Size([894, 64])
hidden: torch.Size([846, 64])
hidden: torch.Size([802, 64])
hidden: torch.Size([910, 64])
hidden: torch.Size([852, 64])
hidden: torch.Size([794, 64])
hidden: torch.Size([867, 64])
hidden: torch.Size([889, 64])
hidden: torch.Size([893, 64])
hidden: torch.Size([846, 64])
hidden: torch.Size([757, 64])
hidden: torch.Size([891, 64])
hidden: torch.Size([116, 64])
hidden: torch.Size([883, 64])
hidden: torch.Size([842, 64])
hidden: torch.Size([980, 64])
hidden: torch.Size([798, 64])
hidden: torch.Size([782, 64])
hidden: torch.Size([851, 64])
hidden: torch.Size([868, 64])
hidden: torch.Size([785, 64])
hidden: torch.Size([814, 64])
hidden: torch.Size([845, 64])
hidden: torch.Size([923, 64])
hidden: torch.Size([881, 64])
hidden: torch.Size([832, 64])
hidden: torch.Size([856, 64])
hidden: tor

hidden: torch.Size([855, 64])
hidden: torch.Size([832, 64])
hidden: torch.Size([795, 64])
hidden: torch.Size([833, 64])
hidden: torch.Size([908, 64])
hidden: torch.Size([936, 64])
hidden: torch.Size([820, 64])
hidden: torch.Size([814, 64])
hidden: torch.Size([848, 64])
hidden: torch.Size([86, 64])
hidden: torch.Size([905, 64])
hidden: torch.Size([824, 64])
hidden: torch.Size([869, 64])
hidden: torch.Size([921, 64])
hidden: torch.Size([784, 64])
hidden: torch.Size([898, 64])
hidden: torch.Size([822, 64])
hidden: torch.Size([852, 64])
hidden: torch.Size([805, 64])
hidden: torch.Size([877, 64])
hidden: torch.Size([803, 64])
hidden: torch.Size([906, 64])
hidden: torch.Size([837, 64])
hidden: torch.Size([805, 64])
hidden: torch.Size([89, 64])
hidden: torch.Size([848, 64])
hidden: torch.Size([910, 64])
hidden: torch.Size([814, 64])
hidden: torch.Size([816, 64])
hidden: torch.Size([846, 64])
hidden: torch.Size([825, 64])
hidden: torch.Size([788, 64])
hidden: torch.Size([906, 64])
hidden: torc

hidden: torch.Size([834, 64])
hidden: torch.Size([819, 64])
hidden: torch.Size([783, 64])
hidden: torch.Size([860, 64])
hidden: torch.Size([832, 64])
hidden: torch.Size([846, 64])
hidden: torch.Size([874, 64])
hidden: torch.Size([932, 64])
hidden: torch.Size([843, 64])
hidden: torch.Size([930, 64])
hidden: torch.Size([852, 64])
hidden: torch.Size([831, 64])
hidden: torch.Size([822, 64])
hidden: torch.Size([101, 64])
hidden: torch.Size([910, 64])
hidden: torch.Size([836, 64])
hidden: torch.Size([795, 64])
hidden: torch.Size([945, 64])
hidden: torch.Size([868, 64])
hidden: torch.Size([870, 64])
hidden: torch.Size([943, 64])
hidden: torch.Size([838, 64])
hidden: torch.Size([851, 64])
hidden: torch.Size([764, 64])
hidden: torch.Size([908, 64])
hidden: torch.Size([805, 64])
hidden: torch.Size([788, 64])
hidden: torch.Size([807, 64])
hidden: torch.Size([69, 64])
hidden: torch.Size([818, 64])
hidden: torch.Size([750, 64])
hidden: torch.Size([887, 64])
hidden: torch.Size([884, 64])
hidden: tor